In [1]:
import os

import torch
from torch import nn
from torch.autograd import Variable

from torchvision import transforms
from torchvision.utils import save_image

import numpy as np
from torch.utils.data import Dataset, DataLoader
if not os.path.exists('./training_img'):os.mkdir('./training_img')

# custom dataloader for .npy file
class numpyDataset(Dataset):
    def __init__(self, data, transform=None):
        
        self.data = torch.from_numpy(data).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x
    
    def __len__(self):
        return len(self.data)


def to_img(x):
    x = x.view(x.size(0), 1, 120, 80)
    return x

num_epochs = 300
batch_size = 100
learning_rate = 1e-3

def add_noise(img):
    noise = torch.randn(img.size()) * (0.1**0.5)
    noisy_img = img + noise
    return noisy_img

def plot_sample_img(img, name):
    img = img.view(1, 120, 80)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)


img_transform = transforms.Compose([
    #transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

traindata       = np.load('./train.npy')
traindataset    = numpyDataset(traindata, img_transform)
trainloader     = DataLoader(traindataset, batch_size=batch_size, shuffle=True)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(120 * 80, 3200),
            nn.ReLU(True),
            nn.Linear(3200, 800),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(800, 3200),
            nn.ReLU(True),
            nn.Linear(3200, 120 * 80),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

loss_train_mean = []
loss_train_std = []

for epoch in range(num_epochs):
    loss_train  = []
    for data in trainloader:
        img = data
        img = img.view(img.size(0), -1)
        noisy_img = add_noise(img)
        noisy_img = Variable(noisy_img)
        img = Variable(img)
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #배치 사이즈 100 이고 전체 사진이 4400개이므로 포문 44번돔
        loss_train_batch    = MSE_loss.item() / len(noisy_img)
        loss_train.append(loss_train_batch)
    # ===================log========================
    loss_train_m     = np.mean(loss_train)
    loss_train_s      = np.std(loss_train)
    #print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data, MSE_loss.data))
    print('epoch [{}/{}], loss:(training) {:.10f}'.format(epoch + 1, num_epochs, loss_train_m))
    loss_train_mean.append(loss_train_m)
    loss_train_std.append(loss_train_s)
    
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        x_noisy = to_img(noisy_img.cpu().data)
        weights = to_img(model.encoder[0].weight.cpu().data)
        save_image(x, './training_img/x_{}.png'.format(epoch))
        save_image(x_hat, './training_img/x_hat_{}.png'.format(epoch))
        save_image(x_noisy, './training_img/x_noisy_{}.png'.format(epoch))
        save_image(weights, './filters/epoch_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_dautoencoder.pth')

testdata       = np.load('./test.npy')
testdataset    = numpyDataset(traindata, img_transform)
testloader     = DataLoader(traindataset, batch_size=400, shuffle=True)

count = 0
for data in testloader:
    img = data
    img = img.view(img.size(0), -1)
    img = Variable(img)
    # ===================forward=====================
    output = model(img)
    #=====
    x = to_img(img.cpu().data)
    x_hat = to_img(output.cpu().data)
    save_image(x, './testing_img/x_{}.png'.format(count))
    save_image(x_hat, './testing_img/x_hat_{}.png'.format(count))
    count += 1

#the submit_file.shape must be (400,1,120,80)

submit_file = x_hat.detach().numpy()
np.save('./younkyoung20172903.npy', submit_file)


epoch [1/1000], loss:0.0054603482, MSE_loss:0.1400176138
epoch [2/1000], loss:0.0039312937, MSE_loss:0.1204736084
epoch [3/1000], loss:0.0036951370, MSE_loss:0.1198438033
epoch [4/1000], loss:0.0034789104, MSE_loss:0.1027563363
epoch [5/1000], loss:0.0033985209, MSE_loss:0.1008602530
epoch [6/1000], loss:0.0033136963, MSE_loss:0.1148618832
epoch [7/1000], loss:0.0032461168, MSE_loss:0.0897171423
epoch [8/1000], loss:0.0031679833, MSE_loss:0.0956600010
epoch [9/1000], loss:0.0030784953, MSE_loss:0.0994745418
epoch [10/1000], loss:0.0030323112, MSE_loss:0.0907550529
epoch [11/1000], loss:0.0029792655, MSE_loss:0.1011105254
epoch [12/1000], loss:0.0029523907, MSE_loss:0.0827395394
epoch [13/1000], loss:0.0029024029, MSE_loss:0.0940068439
epoch [14/1000], loss:0.0028605318, MSE_loss:0.0885569826
epoch [15/1000], loss:0.0028194491, MSE_loss:0.0921012163
epoch [16/1000], loss:0.0027969472, MSE_loss:0.0819711462
epoch [17/1000], loss:0.0027742667, MSE_loss:0.0776667446
epoch [18/1000], loss:0

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(loss_train_mean,label = "training loss")
plt.xlabel('iteration')
plt.ylabel('loss') 
plt.title('Loss')
plt.legend() 
plt.show() 

plt.plot(loss_train_std,label = "loss std")
plt.title('Training standard deviation')
plt.xlabel('iteration')
plt.show()